# 문제정의
## 고객별 신용등급 분류
## Column : Credit_Score - 타깃 변수(y), 신용등급(1=Bad, 2=Standard, 3=Good)
## 타깃(라벨, y값) : Credit_Score

In [3]:
#----------------------
# 라이브러리 가져오기
#----------------------

In [4]:
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')



In [5]:
#----------------------
# EDA(데이터 전처리)
#----------------------

In [6]:
train.head()
train.tail()
train.sample(5)

# Credit_score가 인코딩이 되어있는 상태라 y값을 따로 정의할 필요는 없을 듯!

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Score,Credit_Mix,...,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
4292,27.0,17.0,6.0,28.235992,44.0,Yes,109.067178,311.517379,1,Bad,...,22.0,44320.68,6.0,7.0,16.0,5.0,3433.390000,7.10,2760.28,137.897170
9019,7.0,1.0,1.0,37.065502,291.0,No,105.805305,315.688811,3,Good,...,53.0,15617.35,0.0,2.0,1.0,3.0,1579.445833,0.57,258.68,26.450468
4512,21.0,22.0,4.0,33.462289,224.0,Yes,755.980807,139.542193,2,Standard,...,27.0,69806.76,3.0,4.0,19.0,0.0,6055.230000,5.49,192.43,0.000000
1000,5.0,6.0,0.0,28.820554,397.0,No,345.609135,791.733615,3,Good,...,20.0,106733.13,4.0,4.0,1.0,0.0,8873.427500,0.86,76.23,0.000000
3182,11.0,12.0,1.0,30.905228,276.0,No,659.197695,465.946751,2,Standard,...,50.0,122713.60,5.0,6.0,14.0,2.0,10114.133333,10.46,128.33,146.268887


In [7]:
train.shape, test.shape

((10000, 21), (10000, 20))

In [8]:
# train의 y값 확인
train['Credit_Score']

# 표형태로 볼려면
train[['Credit_Score']]

# 얼마나 카운팅되어있는지 확인
train[['Credit_Score']].value_counts()

Credit_Score
2               5237
1               2978
3               1785
Name: count, dtype: int64

In [9]:
# 범주형/수치형 구성 확인
train.info()
test.info()
# --> 결측치 X, 결측치 처리 필요 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Delay_from_due_date       10000 non-null  float64
 1   Num_of_Delayed_Payment    10000 non-null  float64
 2   Num_Credit_Inquiries      10000 non-null  float64
 3   Credit_Utilization_Ratio  10000 non-null  float64
 4   Credit_History_Age        10000 non-null  float64
 5   Payment_of_Min_Amount     10000 non-null  object 
 6   Amount_invested_monthly   10000 non-null  float64
 7   Monthly_Balance           10000 non-null  float64
 8   Credit_Score              10000 non-null  int64  
 9   Credit_Mix                10000 non-null  object 
 10  Payment_Behaviour         10000 non-null  object 
 11  Age                       10000 non-null  float64
 12  Annual_Income             10000 non-null  float64
 13  Num_Bank_Accounts         10000 non-null  float64
 14  Num_Cre

In [10]:
# 이상치 확인
train.describe()
test.describe()

# 범주형 이상치 확인
train.describe(include='O')
test.describe(include='O')

,Payment_of_Min_Amount,Credit_Mix,Payment_Behaviour
count,10000,10000,10000
unique,3,3,6
top,Yes,Standard,Low_spent_Small_value_payments
freq,5167,4590,3498


In [17]:
# 라벨(정답, y값) 분리
# 학습시켜야하는 모델의 파라미터로 전달
# y_train = train.pop("Credit_Score")
y_train

y_train.value_counts()

Credit_Score
2    5237
1    2978
3    1785
Name: count, dtype: int64

In [12]:
# 기존 레이블 인코딩

from sklearn.preprocessing import LabelEncoder
cols = train.columns[train.dtypes == object] # 범주형 -> 수치형

for col in cols:
    le = LabelEncoder()

    #피팅되는 경우가 안 맞는 경우 각 행마다 이동할 때 train,text행을 합치기
    le.fit(pd.concat([train[col],test[col]] ,axis=0)) # axis=0 행합치기, axis=1 열합치기

    #따로 분리
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

In [14]:
#문자형 -> 숫자형으로 변환
train
test

,Delay_from_due_date,Num_of_Delayed_Payment,Num_Credit_Inquiries,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Amount_invested_monthly,Monthly_Balance,Credit_Mix,Payment_Behaviour,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Monthly_Inhand_Salary,Changed_Credit_Limit,Outstanding_Debt,Total_EMI_per_month
0,4.0,8.0,2.0,33.444469,239.0,1,104.467859,503.976904,1,5,54.0,43754.320,5.0,7.0,7.0,1.0,3938.193333,7.55,515.73,35.374569
1,31.0,19.0,14.0,27.762143,93.0,2,149.137974,219.372773,0,2,40.0,18836.230,6.0,8.0,16.0,6.0,1280.685833,14.26,1310.56,49.557837
2,55.0,16.0,17.0,38.985106,179.0,2,21.862086,299.525207,2,5,27.0,9360.515,8.0,6.0,15.0,5.0,531.042917,11.92,2113.29,21.716999
3,47.0,20.0,8.0,25.577407,93.0,2,170.009215,161.283266,0,5,34.0,30386.200,7.0,8.0,21.0,9.0,2650.183333,17.53,4902.83,193.725852
4,26.0,11.0,7.0,36.580166,207.0,2,296.552469,177.231449,2,1,36.0,49187.070,6.0,6.0,25.0,6.0,3823.922500,12.04,1357.26,198.608332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,27.0,12.0,4.0,38.977305,383.0,2,39.695906,472.869011,2,1,32.0,38839.220,8.0,4.0,19.0,3.0,3272.601667,1.83,1203.51,64.695250
9996,22.0,19.0,2.0,25.582411,363.0,2,89.540202,360.819998,2,1,51.0,24251.490,8.0,4.0,14.0,1.0,1991.957500,10.53,1062.17,18.835550
9997,20.0,15.0,11.0,26.764418,233.0,2,53.574209,311.419464,0,5,45.0,15806.840,6.0,5.0,21.0,3.0,1140.236667,3.41,2580.63,29.029993
9998,60.0,19.0,8.0,35.262194,99.0,0,46.902804,389.691164,0,2,40.0,35462.520,8.0,9.0,31.0,4.0,2718.210000,13.14,2371.87,75.227032
